In [4]:
%pip install ragas


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import pandas as pd
import pyarrow as pa
from datasets import Dataset

path = 'evalSet.csv'
df = pd.read_csv(path)
df["contexts"] = df["contexts"].apply(json.loads)
df = df.drop(columns=["evolution_type", "episode_done", "generated_contexts", "metadata"])
print(df.head())
dataset = Dataset.from_pandas(df)

                                            question  \
0  What are the different arcane traditions in wi...   
1  What spells can a Tiefling cast as part of the...   
2  What is the feature associated with the keyphr...   
3  What is the effect of the Relentless Rage feat...   
4          What are the class features of a Warlock?   

                                            contexts  \
0  [Wizard\n======\nArcane Traditions\n----------...   
1  [Races\n=====\nTiefling\n--------\n### Tieflin...   
2  [Wizard\n======\nArcane Traditions\n----------...   
3  [Barbarian\n=========\nClass Features\n-------...   
4  [Warlock\n=======\nClass Features\n-----------...   

                                        ground_truth  \
0  The most common arcane traditions in wizardry ...   
1  The Tiefling can cast the thaumaturgy cantrip,...   
2  The feature associated with the keyphrase "Ele...   
3  Starting at 11th level, the Relentless Rage fe...   
4  As a warlock, you gain the following class 

In [6]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import RunConfig, evaluate
import nest_asyncio
nest_asyncio.apply()


run_config = RunConfig()
run_config.timeout = 120
run_config.thread_timeout = 120
run_config.max_workers = 4

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    run_config=run_config,
    is_async=False,
)

result
resultPandas = result.to_pandas()
display(resultPandas)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
resultPandas.to_csv("result.csv", index=False)
result
# [0.9313194444180828, 0.925, 0.908733879753922, 0.8966666666666667]
# {'context_precision': 0.9383, 'faithfulness': 0.9750, 'answer_relevancy': 0.9098, 'context_recall': 0.9025}
# {'context_precision': 0.9373, 'faithfulness': 0.9750, 'answer_relevancy': 0.9094, 'context_recall': 0.8925}
# {'context_precision': 0.9433, 'faithfulness': 0.9474, 'answer_relevancy': 0.9082, 'context_recall': 0.9025}
# {'context_precision': 0.9401, 'faithfulness': 0.9750, 'answer_relevancy': 0.9120, 'context_recall': 0.9025}
# {'context_precision': 0.9481, 'faithfulness': 0.9150, 'answer_relevancy': 0.9068, 'context_recall': 0.8811}

# {'context_precision': 0.8856, 'faithfulness': 0.8379, 'answer_relevancy': 0.9451, 'context_recall': 0.7946}
# {'context_precision': 0.8942, 'faithfulness': 0.8507, 'answer_relevancy': 0.9427, 'context_recall': 0.7726}


{'context_precision': 0.8942, 'faithfulness': 0.8507, 'answer_relevancy': 0.9427, 'context_recall': 0.7726}